<a href="https://colab.research.google.com/github/RoisulIslamRumi/MNIST-PyTorch/blob/main/MNIST_custom_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   List item
2.   List item



# Step 1: Load the MNIST dataset

In [11]:
import torch
import torchvision.datasets as ds
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.transforms import ToTensor


In [12]:
train = ds.MNIST(root='./data', train=True, download=True, transform=ToTensor())
test = ds.MNIST(root='./data', train=False, download=True,transform=ToTensor())




In [13]:
print(train.data.size())
print(test.data.size())
print("Label size", train.targets.size())

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])
Label size torch.Size([60000])


# Step 2: Make the dataset iterable

In [14]:
#Load Dataset into Dataloader
batch_size = 60
n_iters = len(train)/batch_size
print("Number of iterations:", n_iters)
num_epochs = 5

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)

test_loader =torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

Number of iterations: 1000.0


In [15]:
#define the class and initialize with all the layers
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()

        #conv1
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu1 = nn.ReLU()
        #Maxpool1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        #conv2
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        #Maxpool2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        #FC layer           #calculated using output formula for conv and pool
                        #channel h *w
        self.fc1 = nn.Linear(32 * 7 * 7, 10)
        #forward pass

    def forward(self, x):
        
        #conv1
        out = self.conv1(x)
        out = self.relu1(out)

        #Maxpool1
        out = self.maxpool1(out)
       
        #conv2
        out = self.conv2(out)
        out = self.relu2(out)

        #maxpool2
        out = self.maxpool2(out)
        
        out = out.view(out.size(0), -1)
        
        #Linear function
        out = self.fc1(out)

        return out



# Step 4: Instantialte Model Class

In [16]:
model = CustomCNN()

# Step 5: Instantialte Loss Class


*   Feedforward Neural Network: Cross Entropy Loss
*   Logistic Regression: Cross Entropy Loss
* Linear Regression: MSE



In [17]:
#cross entropy loss
criterion = nn.CrossEntropyLoss()

# Step 6: Instantialte Optimizer Class

In [18]:
#optimizer
lr = 0.01
optimizer = optim.SGD(model.parameters(), lr=lr)

In [19]:
print(model.parameters())

print(len(list(model.parameters())))

# Convolution 1: 16 Kernels
print(list(model.parameters())[0].size())

# Convolution 1 Bias: 16 Kernels
print(list(model.parameters())[1].size())

# Convolution 2: 32 Kernels with depth = 16
print(list(model.parameters())[2].size())

# Convolution 2 Bias: 32 Kernels with depth = 16
print(list(model.parameters())[3].size())

# Fully Connected Layer 1
print(list(model.parameters())[4].size())

# Fully Connected Layer Bias
print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x7f11312492d0>
6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


#Step 7: Train the Model

In [20]:
iter = 0
loss_list = []
iteration_list = []
accuracy_list = []
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        #load images
        images = images.requires_grad_()

        #clear grads w.r.t. params
        optimizer.zero_grad()

        #Forward pass to get output/logits
        outputs = model(images)

        #calculate loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        #back pass
        loss.backward()
        
        #update param
        optimizer.step()

        iter += 1
        
        if iter % 50 == 0:
            #calc accuracy
            correct = 0
            total = 0
            #iterate test data
            for images,labels in test_loader:
                images = images.requires_grad_()
                outputs = model(images)
                #get predictions from the max val
                _, predicted = torch.max(outputs.data, 1)

                #total labels
                total += labels.size(0)

                #total correct predictions
                correct += (predicted == labels).sum()
            
            acc = 100 * (correct / total)
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(iter)
            accuracy_list.append(acc)
        if iter % 500 == 0:
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), acc))



KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
# visualization loss 
plt.plot(iteration_list,loss_list)
plt.xlabel("Number of iteration")
plt.ylabel("Loss")
plt.title("CNN: Loss vs Number of iteration")
plt.show()

# visualization accuracy 
plt.plot(iteration_list,accuracy_list,color = "red")
plt.xlabel("Number of iteration")
plt.ylabel("Accuracy")
plt.title("CNN: Accuracy vs Number of iteration")
plt.show()